In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imageio
import sqlite3
import math
import os
import json
import urllib
import datetime as dt

from matplotlib import image
from flask import g, Flask, render_template, request, session, redirect
from werkzeug.wrappers import Request, Response
from flask import Flask, render_template, request, jsonify
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

from werkzeug.serving import run_simple

In [19]:
objects = json.load(open("objects_nlp.json"))

In [20]:
con = sqlite3.connect('rijksstudio.db')

In [21]:
cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE IF NOT EXISTS Clicks
               (painting_id INTEGER, 
                object_name TEXT, 
                user_id INTEGER,
                x INTEGER, 
                y INTEGER
                )''')

cur.execute("""CREATE TABLE IF NOT EXISTS Users
               (user_id INTEGER PRIMARY KEY,
                username TEXT,
                password TEXT
                )""")

# Save the changes
con.commit()
con.close()

In [22]:
# Connect to the database and setup the app
app = Flask(__name__)
app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'

def get_db():
    # Connect to the database
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect("rijksstudio.db")
    return db

@app.teardown_appcontext
def close_connection(exception):
    # Close the database
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

In [23]:
@app.route('/', methods=["GET", "POST"])
def index():
    # If logged in, show a painting
    if "user_id" in session:
        # Choose random painting, but make sure there are objects in it
        items = []
        while not items:
            painting = np.random.choice(os.listdir("static/images/paintings/"))
            try:
                items = objects[painting.split(".")[0]]
            except KeyError:
                pass
        
        
        session['all_objects'] = items
        
        obj = np.random.choice(session['all_objects'])
        session["all_objects"].remove(obj) 
        
        session['painting'] = f"static/images/paintings/{painting}"
        session["object"] = obj
        
        session["count"] = 0
        session["score"] = 0
        
        
        # Read image for dimensions
        dims = Image.open(session['painting']).size
        
        # Scaling
        ratio = min(1280 / dims[0], 720 / dims[1])
        session["ratio"] = ratio
                
        # Show the image on an html page
        return render_template('test_jim.html', 
                               name = obj, 
                               url = f'static/images/paintings/{painting}', 
                               width = dims[0] * ratio, 
                               height = dims[1] * ratio)
    
    else: # Else, redirect the user to the login page
        return redirect("/login")

In [24]:
@app.route("/register", methods=["GET", "POST"])
def register():
    # If getting the page, show the html
    if request.method == "GET":
        return render_template("register.html")
    else: # if posting the page, store the data
        username = request.form.get("username")
        password = request.form.get("password")
        
        cur = get_db().cursor()   
        q = cur.execute(f"""INSERT INTO Users
                           VALUES (null, '{username}', '{password}')
                           """)
        # Commit changes
        get_db().commit()

        return redirect("/login")

In [25]:
@app.route("/login", methods=["GET", "POST"])
def login():
    """Compare a username and password against the database and let the user log in if both are correct."""

    # forget any user_id
    session.clear()
    cur = get_db().cursor()
    
    # If posting, check input and attempt login
    if request.method == "POST":
        if not request.form.get("username") or not request.form.get("password"):
            return "please fill in everything"
                
        # Query for the user logging in 
        row = cur.execute(f"""SELECT user_id, password 
                               FROM Users 
                               WHERE username = '{request.form.get('username')}'""").fetchone()
            
        # If user not found or password does not match, throw an error
        if not row or not request.form.get("password") == row[1]:
            return "password error"

        # remember which user has logged in
        session["user_id"] = row[0]
        
        # redirect user to home page
        return redirect("/")

    else: # If getting, display html
        return render_template("login.html")

In [26]:
@app.route("/calculate_score", methods=["POST"])
def calculate_score():
    r = request.get_json()
    
    x = r["x"]
    y = r["y"]
    
    x *= 1 / session["ratio"]
    y *= 1 / session["ratio"]
    
    time = r["time"]
        
    # Load in the image
    img = np.array(Image.open(session['painting']))
    dims = Image.open(session['painting']).size
        
    plot, box, points = create_plot(img, x, y)
    
    if len(points) < 3:
        try:
            obj = np.random.choice(session["all_objects"])
            session["object"] = obj
            session["all_objects"].remove(obj)
        except:
            return jsonify({"msg" : "You found all the objects in the painting!", "url" : session["painting"], "obj": "No objects left!", "total_score": session["score"]})
               
        session["count"] += 1
        session["score"] += 100
        
        return jsonify({"msg" : "Not enough data - You scored 100 points!", "url" : plot, "obj": obj, "total_score": session["score"]})
       
    if isinstance(box, tuple):
        return determine_score(box, x, y, time, plot)[1]
    else:
        result = (0, None)

        for b in box:
            current = determine_score(b, x, y, time, plot)
            if current[0] >= result[0]:
                result = current
                
        return result[1]

In [27]:
def normalize(points):
    # Convert clicks to array
    a = np.array(points)    
    # Calculate mean and standard deviation
    mean, stdev = np.mean(a, axis = 0), np.std(a, axis=0)
    
    # Normalize values, and see if they are more than a quarter standard deviation 
    # apart from the mean
    outliers = ((np.abs(a[:,0] - mean[0]) > 0.25 * stdev[0])
               * (np.abs(a[:,1] - mean[1]) > 0.25 * stdev[1]))
        
    return a[~outliers]

In [28]:
def find_edges(points):
    """Find the edges of the user clicks to determine the boxes"""
    min_x = min(points, key = lambda x: x[0])[0]
    min_y = min(points, key = lambda x: x[1])[1]

    max_x = max(points, key = lambda x: x[0])[0]
    max_y = max(points, key = lambda x: x[1])[1]
    
    return min_x, min_y, max_x, max_y

In [29]:
def determine_clusters(points):
    # Convert to array
    a = np.array(points)
    
    # Assume no. clusters is 1 by default
    final_n = 1
    max_silhouette = 0.55  
        
    for n in range(2, 6):
        # Fit kmeans and calculate silhouette score
        kmeans = KMeans(n_clusters= n)    
        labels = kmeans.fit_predict(a)
        silhouette_avg = silhouette_score(a, labels)
                
        # If this silhouette score is better than all others
        # set the current no. clusters to be the result
        if silhouette_avg > max_silhouette:
            max_silhouette = silhouette_avg
            final_n = n
    
    # Using the best found n, cluster all data
    kmeans = KMeans(n_clusters = final_n)
    df = pd.DataFrame(a, columns = ["X", "Y"])
    df["label"] = kmeans.fit_predict(df)
    
    return df, final_n

In [30]:
def create_box(points):
    min_x, min_y, max_x, max_y = find_edges(points)
    box = (min_x, max_x, min_y, max_y)    
    return box

In [31]:
def shaver(points):
    """Shaves outliers off an array"""
    
    df = pd.DataFrame(points)
    to_shave = int(len(df) * 0.05)
        
    if to_shave >= 2:
        df = df.sort_values(0)[to_shave // 2:-to_shave // 2]
        df = df.sort_values(1)[to_shave // 2:-to_shave // 2]
    
    return df.sort_index().values

In [32]:
def create_plot(img, x, y):   
    
    # Query the database for relevant clicks
    cur = get_db().cursor()   
    q = cur.execute(f"""SELECT x, y
                        FROM Clicks 
                        WHERE painting_id == '{session["painting"].split("/")[-1]}'
                        AND object_name == '{session["object"].replace(" ", "_")}'""")   
           
    # Create border edges
    points = [(r[0], r[1]) for r in q]    

    # plot image
    plt.axis('off')
    plt.imshow(img)
        
    # If there are at least three clicks, we can plot borders
    if len(points) > 2: 
        # If we only have a few clicks, do not make clusters yet
        points = shaver(points)
        
        if len(points) < 10:
            
            box = create_box(points)            
            min_x, max_x, min_y, max_y = box
            v_lines = [min_x, max_x]
            h_lines = [min_y, max_y]
            
            # Plot border
            for line in v_lines:
                plt.vlines(x=line, color="r", ymin=min_y, ymax=max_y,
                           ls='-', lw=0.5)

            for line in h_lines:
                plt.hlines(y=line, color="r", xmin=min_x, xmax=max_x,
                           ls='-', lw=0.5)
            
            plt.scatter([i[0] for i in points], [i[1] for i in points], c="b", s=0.25)
        else:
            df, n_clusters = determine_clusters(points)            
            boxes = []
            
            for i in range(n_clusters):
                points = df[df["label"] == i][["X", "Y"]].values
                
                box = create_box(points)            
                boxes.append(box)
                min_x, max_x, min_y, max_y = box
                v_lines = [min_x, max_x]
                h_lines = [min_y, max_y]
    
                # Plot borders
                for line in v_lines:
                    plt.vlines(x=line, color="r", ymin=min_y, ymax=max_y,
                               ls='-', lw=0.5)

                for line in h_lines:
                    plt.hlines(y=line, color="r", xmin=min_x, xmax=max_x,
                               ls='-', lw=0.5)
                    
                plt.scatter([i[0] for i in points], [i[1] for i in points], c="b", s=0.25)
                
            box = boxes
    else:
        box = None
    
    ob = session["object"].replace(" ", "_")
    paint = session["painting"].split("/")[-1].replace(".png", "")
    
    loc = f'static/images/{paint}_{ob}_{dt.datetime.today().timestamp()}.png'
    
    # Plot where the user clicked
    plt.scatter(x, y, c="r", s=0.5)
    
    # Store the rendered figure    
    plt.savefig(loc,
                bbox_inches='tight',
                dpi = 250, # set to like 500 if we want very pretty high quality paintings that take forever to load
                pad_inches = 0) 

    plt.close()
    
    # Store click in database
    cur.execute(f"""INSERT INTO Clicks
                    VALUES ("{session["painting"].split("/")[-1]}", 
                            "{session["object"].replace(" ", "_")}", 
                            "{session["user_id"]}", 
                            {x}, 
                            {y})""")
    
    get_db().commit()
    
    return loc, box, points

In [33]:
def determine_score(box, x, y, time, plot):
    # max score is 100
    score = 100
    
    # score goes down by 25 per second
    score -= time * 2.5
    
    # extract box edges
    min_x, max_x, min_y, max_y = box

    # find the biggest box size
    size = max(max_x - min_x, max_y - min_y)

    
    try:
        obj = np.random.choice(session["all_objects"])
        session["object"] = obj
        session["all_objects"].remove(obj)
    except:
        return jsonify({"msg" : "You found all the objects in the painting!", "url" : session["painting"], "obj": "No objects left!", "total_score": session["score"]})
           
    session["count"] += 1
    
    
    # If the click is within the box, give full points
    if (min_x < x < max_x) and (min_y < y < max_y):
        session["score"] += 100
        return 100, jsonify({"msg" : "Inside box - You scored 100 points", "url" : plot, "obj": obj, "total_score": session["score"]})
    else:
        # distance to closest point on x-axis
        dx = max(min_x - x, 0, x - max_x)
        # distance to closest point on y-axis
        dy = max(min_y - y, 0, y - max_y)
        # pythagoras theorem
        distance = np.sqrt(dx**2 + dy**2)     
        
        if distance > size:
            session["score"] += 0
            return 0, jsonify({"msg" : "Too far - You scored 0 points!", "url" : plot, "obj": obj, "total_score": session["score"]})
        else:
            final_score = math.exp(-4 * (distance / size)) * score
            session["score"] += final_score
            return final_score, jsonify({"msg": f"You scored {final_score} points!", "url" : plot, "obj": obj, "total_score": session["score"]})

In [ ]:
run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Jan/2022 14:12:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:12:37] "GET /static/images/paintings/SK-A-137.png HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:12:40] "POST /calculate_score HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:12:40] "GET /static/images/SK-A-137_wine_1642511559.636403.png HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:12:42] "POST /calculate_score HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:12:43] "GET /static/images/SK-A-137_silver_1642511561.862456.png HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:12:45] "POST /calculate_score HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:12:45] "GET /static/images/SK-A-137_fruits_1642511564.481706.png HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:12:47] "POST /calculate_score HTTP/1.1" 200 -
